In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cd /content/drive/MyDrive/UAV/Model/data

In [3]:
!cp -r /content/drive/MyDrive/UAV/Model/data/yolo_mixed/images/train/*.jpg /content/drive/MyDrive/UAV/Model/data/yolo_mixed/images/train/*.JPG /content/drive/MyDrive/UAV/Model/data/efficientdet/images/train2017/

In [4]:
!cp -r /content/drive/MyDrive/UAV/Model/data/yolo_mixed/images/val/*.jpg /content/drive/MyDrive/UAV/Model/data/yolo_mixed/images/val/*.JPG /content/drive/MyDrive/UAV/Model/data/efficientdet/images/val2017/

In [45]:
# -*- coding: utf-8 -*-
'''
@time: 2019/01/11 11:28
spytensor
'''

import os
import json
import numpy as np
import pandas as pd
import glob
import cv2
import os
import shutil
from IPython import embed
from sklearn.model_selection import train_test_split
np.random.seed(41)

#0为背景
classname_to_id = {"person": 0}

class Csv2CoCo:

    def __init__(self,image_dir,total_annos):
        self.images = []
        self.annotations = []
        self.categories = []
        self.img_id = 0
        self.ann_id = 0
        self.image_dir = image_dir
        self.total_annos = total_annos

    def save_coco_json(self, instance, save_path):
        json.dump(instance, open(save_path, 'w'), ensure_ascii=False, indent=2)  # indent=2 更加美观显示

    # 由txt文件构建COCO
    def to_coco(self, keys):
        self._init_categories()
        if 'filename' in keys:
          keys.remove('filename')
        for key in keys:
            self.images.append(self._image(key))
            shapes = self.total_annos[key]
            for shape in shapes:
                bboxi = []
                for cor in shape[3:7]:
                    if float(cor)==float(cor):
                      bboxi.append(int(float(cor)))
                label = 'person'
                if len(bboxi)!=0:
                  annotation = self._annotation(bboxi,label)
                  self.annotations.append(annotation)
                  self.ann_id += 1
            if len(bboxi)!=0:
              self.img_id += 1
        if len(bboxi)!=0:
          instance = {}
          instance['info'] = 'spytensor created'
          instance['license'] = ['license']
          instance['images'] = self.images
          instance['annotations'] = self.annotations
          instance['categories'] = self.categories
          return instance

    # 构建类别
    def _init_categories(self):
        for k, v in classname_to_id.items():
            category = {}
            category['id'] = v
            category['name'] = k
            self.categories.append(category)

    # 构建COCO的image字段
    def _image(self, path):
        image = {}
        print(path)
        if path!='filename':
          img = cv2.imread(self.image_dir + path)
          image['height'] = img.shape[0]
          image['width'] = img.shape[1]
          image['id'] = self.img_id
          image['file_name'] = path
          return image

    # 构建COCO的annotation字段
    def _annotation(self, shape,label):
        # label = shape[-1]
        points = shape[:4]
        annotation = {}
        annotation['id'] = self.ann_id
        annotation['image_id'] = self.img_id
        annotation['category_id'] = int(classname_to_id[label])
        annotation['segmentation'] = self._get_seg(points)
        annotation['bbox'] = self._get_box(points)
        annotation['iscrowd'] = 0
        annotation['area'] = 1.0
        return annotation

    # COCO的格式： [x1,y1,w,h] 对应COCO的bbox格式
    def _get_box(self, points):
        min_x = points[0]
        min_y = points[1]
        max_x = points[2]
        max_y = points[3]
        return [min_x, min_y, max_x - min_x, max_y - min_y]
    # segmentation
    def _get_seg(self, points):
        min_x = points[0]
        min_y = points[1]
        max_x = points[2]
        max_y = points[3]
        h = max_y - min_y
        w = max_x - min_x
        a = []
        a.append([min_x,min_y, min_x,min_y+0.5*h, min_x,max_y, min_x+0.5*w,max_y, max_x,max_y, max_x,max_y-0.5*h, max_x,min_y, max_x-0.5*w,min_y])
        return a
   

if __name__ == '__main__':
    csv_file = "/content/drive/MyDrive/UAV/Model/data/mixedImg_info.csv"
    image_dir = "/content/drive/MyDrive/UAV/Model/data/images_mixed/"
    saved_coco_path = "/content/drive/MyDrive/UAV/Model/data/efficientdet/"
    # 整合csv格式标注文件
    total_csv_annotations = {}
    annotations = pd.read_csv(csv_file,header=None).values
    for annotation in annotations:
        key = annotation[0].split(os.sep)[-1]
        value = np.array([annotation[1:]])
        if key in total_csv_annotations.keys():
            total_csv_annotations[key] = np.concatenate((total_csv_annotations[key],value),axis=0)
        else:
            total_csv_annotations[key] = value
    # 按照键值划分数据
    total_keys = list(total_csv_annotations.keys())
    train_keys, val_keys = train_test_split(total_keys, test_size=0.2)
    print("train_n:", len(train_keys), 'val_n:', len(val_keys))
    # 创建必须的文件夹
    if not os.path.exists('%scoco/annotations/'%saved_coco_path):
        os.makedirs('%scoco/annotations/'%saved_coco_path)
    if not os.path.exists('%scoco/images/train2017/'%saved_coco_path):
        os.makedirs('%scoco/images/train2017/'%saved_coco_path)
    if not os.path.exists('%scoco/images/val2017/'%saved_coco_path):
        os.makedirs('%scoco/images/val2017/'%saved_coco_path)
    # 把训练集转化为COCO的json格式
    l2c_train = Csv2CoCo(image_dir=image_dir,total_annos=total_csv_annotations)
    train_instance = l2c_train.to_coco(train_keys)
    l2c_train.save_coco_json(train_instance, '%scoco/annotations/instances_train2017.json'%saved_coco_path)
    for file in train_keys:
        if file!='filename':
          shutil.copy(image_dir+file,"%scoco/images/train2017/"%saved_coco_path)
    for file in val_keys:
        if file!='filename':
          shutil.copy(image_dir+file,"%scoco/images/val2017/"%saved_coco_path)
    # 把验证集转化为COCO的json格式
    l2c_val = Csv2CoCo(image_dir=image_dir,total_annos=total_csv_annotations)
    val_instance = l2c_val.to_coco(val_keys)
    l2c_val.save_coco_json(val_instance, '%scoco/annotations/instances_val2017.json'%saved_coco_path)

train_n: 632 val_n: 159
681.JPG
041.jpg
424.JPG
541.JPG
602.JPG
393.JPG
292.jpg
164.jpg
662.JPG
634.JPG
760.JPG
446.JPG
261.jpg
082.jpg
744.JPG
367.JPG
295.jpg
323.jpg
599.JPG
002.jpg
595.JPG
297.jpg
034.jpg
668.JPG
097.jpg
659.JPG
517.JPG
666.JPG
364.JPG
384.JPG
642.JPG
622.JPG
479.JPG
387.JPG
445.JPG
149.jpg
773.JPG
293.jpg
729.JPG
722.JPG
318.jpg
301.jpg
682.JPG
651.JPG
166.jpg
043.jpg
695.JPG
611.JPG
770.JPG
582.JPG
706.JPG
536.JPG
257.jpg
751.JPG
491.JPG
059.jpg
155.jpg
294.jpg
213.jpg
247.jpg
346.jpg
653.JPG
324.jpg
368.JPG
726.JPG
485.JPG
513.JPG
616.JPG
231.jpg
258.jpg
062.jpg
427.JPG
766.JPG
279.jpg
755.JPG
151.jpg
298.jpg
612.JPG
463.JPG
702.JPG
061.jpg
345.jpg
028.jpg
107.jpg
392.JPG
665.JPG
644.JPG
456.JPG
299.jpg
543.JPG
680.JPG
223.jpg
638.JPG
448.JPG
154.jpg
769.JPG
211.jpg
222.jpg
467.JPG
609.JPG
048.jpg
259.jpg
632.JPG
572.JPG
057.jpg
386.JPG
138.jpg
763.JPG
524.JPG
012.jpg
248.jpg
132.jpg
562.JPG
561.JPG
649.JPG
047.jpg
641.JPG
359.JPG
787.JPG
705.JPG
788.JPG
784.JPG
